In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import sys
from google.colab import drive
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.autograd import Function
import argparse
import csv
!nvidia-smi
!pip install torchvision==0.5.0
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
drive.mount('/content/gdrive')

Sat Nov 28 14:21:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !unzip 'gdrive/My Drive/colab/hw3/hw3_data.zip' -d 'gdrive/My Drive/colab/hw3/'
# !unzip 'gdrive/My Drive/colab/hw3/hw3_data.zip'

In [3]:
source_train_label = {}
source_test_label = {}
target_train_label = {}
target_test_label = {}
with open('hw3_data/digits/mnistm/train.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            source_train_label[row[0]]=row[1]
with open('hw3_data/digits/mnistm/test.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            source_test_label[row[0]]=row[1]
with open('hw3_data/digits/svhn/train.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            target_train_label[row[0]]=row[1]
with open('hw3_data/digits/svhn/test.csv',newline='') as csvfile:
    rows = csv.reader(csvfile)
    count = 0
    for row in rows:
        if count == 0:
            count+=1
            continue
        else:
            target_test_label[row[0]]=row[1]

In [4]:
class Allen(Dataset): 
    def __init__(self, transform = None):
        self.transform = transform
        self.common_path = 'hw3_data/digits/mnistm/train/'        
        self.filename = sorted(os.listdir(self.common_path))
        self.len = len(self.filename)

    def __getitem__(self, index):
        path = os.path.join(self.common_path + str(self.filename[index]))
        label = float(source_train_label[self.filename[index]])
        im = Image.open(path)          
        if self.transform is not None:
            im = self.transform(im)

        return im, torch.tensor(label,dtype=torch.long)

    def __len__(self):

        return(self.len)

trainset = Allen(transform = transforms.ToTensor())

train_loader = DataLoader(trainset, batch_size = 64, shuffle = True, drop_last=True)

print('source 大小 : ',len(trainset))

source 大小 :  60000


In [5]:
class Jungle(Dataset): 
    def __init__(self, transform = None):
        self.transform = transform
        self.common_path = 'hw3_data/digits/svhn/train/'
        self.filename = sorted(os.listdir(self.common_path))
        self.len = len(self.filename)

    def __getitem__(self, index):
        path = os.path.join(self.common_path + str(self.filename[index]))
        im = Image.open(path)    
        if self.transform is not None:
            im = self.transform(im)

        return im

    def __len__(self):

        return(self.len)

targetset = Jungle(transform = transforms.ToTensor())

target_loader = DataLoader(targetset, batch_size = 64, shuffle = True, drop_last=True)


print('target 大小 : ',len(targetset))

target 大小 :  73257


In [6]:
class Tony(Dataset): 
    def __init__(self, transform = None):
        self.transform = transform
        self.common_path = 'hw3_data/digits/svhn/test/'
        self.filename = sorted(os.listdir(self.common_path))
        self.len = len(self.filename)

    def __getitem__(self, index):
        path = os.path.join(self.common_path + str(self.filename[index]))
        label = float(target_test_label[self.filename[index]])
        im = Image.open(path)         
        if self.transform is not None:
            im = self.transform(im)

        return im, torch.tensor(label,dtype=torch.long)

    def __len__(self):

        return(self.len)

testset = Tony(transform = transforms.ToTensor())

test_loader = DataLoader(testset, batch_size = 64)


print('test 大小 : ',len(testset))

test 大小 :  26032


In [7]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [8]:
class DANN(nn.Module):
    def __init__(self,F,C,D):
        super(DANN,self).__init__()
        self.F = F
        self.C = C
        self.D = D
    def forward(self, x, alpha=0):
        feature = self.F(x)

        class_out = self.C(feature)

        reverse_feature = ReverseLayerF.apply(feature, alpha)
        domain_out = self.D(reverse_feature)
        return class_out, domain_out

In [9]:
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()

        self.feature = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  #64*28*28
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, 1, 1),  #128*28*28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, 2, 1), #256*12*12
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, 3, 2, 1), #256*10*10
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 512, 3, 2, 1), #512*8*8
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )

        self.class_classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 10),
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512, 1),
        )

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 512)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

#===============================================================================

# class CNNModel(nn.Module):

#     def __init__(self):
#         super(CNNModel, self).__init__()

#         self.feature = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),  #64*28*28
#             nn.BatchNorm2d(64), 
#             nn.ReLU(),
#             nn.MaxPool2d(2),

#             nn.Conv2d(64, 128, 3, 1, 1),  #128*28*28
#             nn.BatchNorm2d(128),
#             nn.ReLU(),

#             nn.Conv2d(128, 256, 3, 2, 1), #256*12*12
#             nn.BatchNorm2d(256),
#             nn.ReLU(),

#             nn.Conv2d(256, 256, 3, 2, 1), #256*10*10
#             nn.BatchNorm2d(256),
#             nn.ReLU(),

#             nn.Conv2d(256, 512, 3, 2, 1), #512*8*8
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#         )

#         self.class_classifier = nn.Sequential(
#             nn.Linear(512*2*2, 512),
#             nn.ReLU(),
#             nn.Dropout(0.5),

#             nn.Linear(512, 10),
#         )

#         self.domain_classifier = nn.Sequential(
#             nn.Linear(512*2*2, 512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(),

#             nn.Linear(512, 1),
#         )

#     def forward(self, input_data, alpha):
#         input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
#         feature = self.feature(input_data)
#         feature = feature.view(-1, 512*2*2)
#         reverse_feature = ReverseLayerF.apply(feature, alpha)
#         class_output = self.class_classifier(feature)
#         domain_output = self.domain_classifier(reverse_feature)

#         return class_output, domain_output

In [10]:
model = CNNModel().to(device)
print(model)

In [11]:
opt = optim.Adam(model.parameters(), lr = 0.001)
loss_class = nn.CrossEntropyLoss()
loss_domain = nn.BCEWithLogitsLoss()

In [12]:
# model = torch.load('gdrive/My Drive/colab/hw3/problem3/U_M/b')
n_epoch = 30
max_epoch = 30
step = 0
best_acc = 0.42
for epoch in range(1, max_epoch+1):
    # len_dataloader = len(train_loader)
    len_dataloader = min(len(train_loader), len(target_loader))
    data_source_iter = iter(train_loader)
    data_target_iter = iter(target_loader)
    model.train()
    step = 0
    for i in range(len_dataloader):

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        data_source = data_source_iter.next()
        s_img, s_label = data_source
        model.zero_grad()

        batch_size = len(s_label)

        domain_label = torch.ones(batch_size)

        s_img, s_label, domain_label = s_img.to(device), s_label.to(device), domain_label.to(device)

        class_output, domain_output = model(s_img, alpha=0.1)
        err_s_label = loss_class(class_output, s_label)
        domain_label = domain_label.view(-1,1)
        err_s_domain = loss_domain(domain_output, domain_label)

        #==================================
        # a.

        # err = err_s_label + err_s_domain

        #==================================
        # training model using target data
        # b.

        data_target = data_target_iter.next()
        t_img = data_target

        batch_size = len(t_img)

        domain_label = torch.zeros(batch_size)

        t_img, domain_label = t_img.to(device), domain_label.to(device)

        _, domain_output = model(t_img, alpha=0.1)
        domain_label = domain_label.view(-1,1)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label

        #==================================

        err.backward()
        opt.step()  

           


    model.eval()
    with torch.no_grad():
        correct = 0
        count = 0
        for batch_idx, data in enumerate(test_loader):
            im, label = data
            im, label = im.to(device), label.to(device)

            c, d = model(im,0)
            c = c.view(-1,10)
            pred = torch.max(c, 1)[1]
            for j in range(len(pred)):
                if pred[j] == label[j]:
                    correct+=1
                count+=1
    print('epoch = ',epoch)
    print('test acc = ',correct/count)
    print('total : ', count)
    print('===========================================')
    temp_acc = correct/count
    if temp_acc > best_acc:
        print('good!!!!!!!!!!!!   the current best acc is : ',temp_acc)
        best_acc = temp_acc
        torch.save(model,'gdrive/My Drive/colab/hw3/problem3/M_S/final')
  # U to M 20.8% 20epoch
  # M to S 34.4% 20epoch
  # S to U 68.9% 20epoch

  # U to M 60.8% 50epoch
  # M to S 54.8% 50epoch
  # S to U 74.4% 100epoch

  # U to M 98.1% 20epoch
  # M to S 92.1% 20epoch
  # S to U 97.3% 20epoch

epoch =  1
test acc =  0.3864090350338045
total :  26032
epoch =  2
test acc =  0.32836508912108175
total :  26032
epoch =  3
test acc =  0.3127304855562385
total :  26032
epoch =  4
test acc =  0.23728488014751076
total :  26032
epoch =  5
test acc =  0.27293331284572836
total :  26032
epoch =  6
test acc =  0.26720958819913954
total :  26032
epoch =  7
test acc =  0.25138291333743085
total :  26032
epoch =  8
test acc =  0.22249539028887522
total :  26032
epoch =  9
test acc =  0.2473878303626306
total :  26032
epoch =  10
test acc =  0.216579594345421
total :  26032
epoch =  11
test acc =  0.2625614628149969
total :  26032
epoch =  12
test acc =  0.27712046711739396
total :  26032
epoch =  13
test acc =  0.2822679778733866
total :  26032
epoch =  14
test acc =  0.27512292562999385
total :  26032
epoch =  15
test acc =  0.21615703749231716
total :  26032
epoch =  16
test acc =  0.2570682237246466
total :  26032
epoch =  17
test acc =  0.26064074984634295
total :  26032
epoch =  18
te

KeyboardInterrupt: ignored

In [ ]:
# torch.save(F,'gdrive/My Drive/colab/hw3/problem3/S_U/F_baseline')
# torch.save(C,'gdrive/My Drive/colab/hw3/problem3/S_U/C_baseline')
# torch.save(D,'gdrive/My Drive/colab/hw3/problem3/S_U/D_baseline')
# torch.save(model,'gdrive/My Drive/colab/hw3/problem3/U_M/b')